In [0]:
#package required by Keras
import tensorflow as tf
import numpy as np
import PIL

from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.applications.mobilenet import decode_predictions
from keras.preprocessing import image
from keras import backend as K


Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def get_input_dim(input_layer):
  data_shape = input_layer.input_shape
  if data_shape[1] == 3: #channel first arrangement
    return (data_shape[2:4])
  else: #channel last arrangement
    return (data_shape[1:3])

def prepare_input(img_file,size): 
  img = image.load_img(img_file, target_size=size)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis = 0)
  return(preprocess_input(x))



In [0]:
my_model = MobileNet(include_top=True, weights='imagenet')
my_model.summary()
layer = my_model.layers[0]
layer.get_config()



In [0]:
!pip install iptcinfo3
from iptcinfo3 import IPTCInfo
import os

#keywords from IPTC need to convert to utf-8
convert_utf8 = lambda x: x.decode('utf-8')

img_path = '/content/drive/My Drive/photos/test_image_tagger'
img_list = os.listdir(img_path)

input_layer = my_model.layers[0]
input_sz = get_input_dim(input_layer)

for img in img_list:
  img_file = os.path.join(img_path,img)

  x = prepare_input(img_file,input_sz)
  preds = my_model.predict(x)
  decoded_preds = decode_predictions(preds, top=3)[0] #get top 3 prediction
  
  
  #retrieve keywords from image
  info = IPTCInfo(img_file)
  
  #init new keyword list with the existing one(s) retreived from the image 
  new_keywords = list(map(convert_utf8, info['keywords']))
  
  for prediction in decoded_preds:
    desc = prediction[1] #2nd element of the prediction is the description
    
    #only append the keyword if not already exist
    if desc not in new_keywords:
      new_keywords.append(desc)

  info['keywords'] = new_keywords
  status_okay = info.save()



